In [3]:
#pip install rouge_score

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24970 sha256=c994e2ae6e470da3894efeef0582f746530fc12fb94978d68bf23e06f20fc854
  Stored in directory: c:\users\administrator.dai-pc2\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [ ]:
# with splitting of the text into sections and then using its summarizes

In [2]:
from transformers import BartTokenizer, BartForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration, LEDTokenizer, LEDForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from collections import Counter
import re

# Initialize models and tokenizers
models = {
    'facebook/bart-large-cnn': (BartTokenizer, BartForConditionalGeneration),
    'google/pegasus-xsum': (PegasusTokenizer, PegasusForConditionalGeneration),
    'google/pegasus-cnn_dailymail': (PegasusTokenizer, PegasusForConditionalGeneration),
    'allenai/led-large-16384': (LEDTokenizer, LEDForConditionalGeneration),
    'allenai/led-large-16384-arxiv': (LEDTokenizer, LEDForConditionalGeneration)
}

def get_model_and_tokenizer(model_name):
    tokenizer_class, model_class = models[model_name]
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name)
    return tokenizer, model

def preprocess_text(text):
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces/newlines with a single space
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

def summarize_text(model_name, text, max_length=1000, min_length=40):
    tokenizer, model = get_model_and_tokenizer(model_name)
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def split_into_sections(text, section_length=1000, overlap=200):
    text = preprocess_text(text)
    sections = []
    start = 0
    while start < len(text):
        end = start + section_length
        sections.append(text[start:end])
        start = end - overlap
    return sections

def summarize_sections(model_name, sections):
    return [summarize_text(model_name, section) for section in sections]

def combine_summaries(summaries):
    combined_summary = "\n\n".join(summaries)
    return combined_summary

def calculate_rouge_scores(reference, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, summary)
    return scores

def calculate_bleu_score(reference, summary):
    reference_tokens = reference.split()
    summary_tokens = summary.split()
    bleu_score = sentence_bleu([reference_tokens], summary_tokens)
    return bleu_score

def calculate_diversity(summary):
    words = summary.split()
    word_counts = Counter(words)
    return len(word_counts) / len(words) if words else 0

def calculate_novelty(summary, reference):
    summary_words = set(summary.split())
    reference_words = set(reference.split())
    novel_words = summary_words - reference_words
    return len(novel_words) / len(summary_words) if summary_words else 0

def main(model_name, text):
    sections = split_into_sections(text)
    summaries = summarize_sections(model_name, sections)
    final_summary = combine_summaries(summaries)
    return final_summary

# Example text and reference (using the full text as a reference here)
text = """
Harry Potter is a series of seven fantasy novels written by British author J. K. Rowling. The novels chronicle the lives of a young wizard, Harry Potter, and his friends, Hermione Granger and Ron Weasley, all of whom are students at Hogwarts School of Witchcraft and Wizardry. The main story arc concerns Harry's conflict with Lord Voldemort, a dark wizard who intends to become immortal, overthrow the wizard governing body known as the Ministry of Magic, and subjugate all wizards and Muggles (non-magical people).

The series was originally published in English by Bloomsbury in the United Kingdom and Scholastic Press in the United States. A series of many genres, including fantasy, drama, coming-of-age fiction, and the British school story (which includes elements of mystery, thriller, adventure, horror, and romance), the world of Harry Potter explores numerous themes and includes many cultural meanings and references.[1] Major themes in the series include prejudice, corruption, madness, and death.[2][3]

Since the release of the first novel, Harry Potter and the Philosopher's Stone, on 26 June 1997, the books have found immense popularity, positive reviews, and commercial success worldwide. They have attracted a wide adult audience as well as younger readers and are widely considered cornerstones of modern literature.[4][5] As of February 2023, the books have sold more than 600 million copies worldwide, making them the best-selling book series in history, and have been available in 85 languages.[6] The last four books consecutively set records as the fastest-selling books in history, with the final instalment selling roughly 2.7 million copies in the United Kingdom and 8.3 million copies in the United States within twenty-four hours of its release.

Warner Bros. Pictures adapted the original seven books into an eight-part namesake film series. In 2016, the total value of the Harry Potter franchise was estimated at $25 billion,[7] making it one of the highest-grossing media franchises of all time. Harry Potter and the Cursed Child is a play based on a story co-written by Rowling.

The success of the books and films has allowed the Harry Potter franchise to expand with numerous derivative works, a travelling exhibition that premiered in Chicago in 2009, a studio tour in London that opened in 2012, a digital platform on which J. K. Rowling updates the series with new information and insight, and a trilogy of spin-off films premiering in November 2016 with Fantastic Beasts and Where to Find Them, among many other developments. Themed attractions, collectively known as The Wizarding World of Harry Potter, have been built at several Universal Destinations & Experiences amusement parks around the world.
"""

# Evaluate each model
results = {}

for model_name in models.keys():
    print(f"Evaluating model: {model_name}")
    # Generate summary
    formatted_summary = main(model_name, text)
    
    # Evaluate summary
    rouge_scores = calculate_rouge_scores(text, formatted_summary)
    bleu_score = calculate_bleu_score(text, formatted_summary)
    diversity_score = calculate_diversity(formatted_summary)
    novelty_score = calculate_novelty(formatted_summary, text)
    
    # Store results
    results[model_name] = {
        'ROUGE Scores': rouge_scores,
        'BLEU Score': bleu_score,
        'Diversity Score': diversity_score,
        'Novelty Score': novelty_score
    }

# Print comparative results
for model_name, metrics in results.items():
    print(f"\nResults for {model_name}:")
    print("ROUGE Scores:", metrics['ROUGE Scores'])
    print("BLEU Score:", metrics['BLEU Score'])
    print("Diversity Score:", metrics['Diversity Score'])
    print("Novelty Score:", metrics['Novelty Score'])


Evaluating model: facebook/bart-large-cnn
Evaluating model: google/pegasus-xsum


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and 

Evaluating model: google/pegasus-cnn_dailymail


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to 

Evaluating model: allenai/led-large-16384


Input ids are automatically padded from 196 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 181 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 198 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 44 to 1024 to be a multiple of `config.attention_window`: 1024


Evaluating model: allenai/led-large-16384-arxiv

Results for facebook/bart-large-cnn:
ROUGE Scores: {'rouge1': Score(precision=0.9662921348314607, recall=0.3830734966592428, fmeasure=0.5486443381180224), 'rouge2': Score(precision=0.8870056497175142, recall=0.35044642857142855, fmeasure=0.5024000000000001), 'rougeL': Score(precision=0.9157303370786517, recall=0.36302895322939865, fmeasure=0.5199362041467305)}
BLEU Score: 0.09473576161461329
Diversity Score: 0.7005649717514124
Novelty Score: 0.3467741935483871

Results for google/pegasus-xsum:
ROUGE Scores: {'rouge1': Score(precision=0.48091603053435117, recall=0.4209354120267261, fmeasure=0.4489311163895487), 'rouge2': Score(precision=0.17346938775510204, recall=0.15178571428571427, fmeasure=0.16190476190476188), 'rougeL': Score(precision=0.25190839694656486, recall=0.22048997772828507, fmeasure=0.2351543942992874)}
BLEU Score: 0.0799063684594486
Diversity Score: 0.23035230352303523
Novelty Score: 0.4235294117647059

Results for google/

In [ ]:
To determine which model performs best, we should consider the various evaluation metrics and their relative importance:

ROUGE Scores
ROUGE-1 measures the overlap of unigrams.
ROUGE-2 measures the overlap of bigrams.
ROUGE-L measures the longest common subsequence.
BLEU Score
Measures the precision of n-grams in the summary relative to the reference text.
Diversity Score
Measures the richness of vocabulary in the summary.
Novelty Score
Measures the proportion of novel words in the summary not present in the reference text.

Comparative Results:

facebook/bart-large-cnn:
ROUGE Scores: Good balance but lower compared to others.
BLEU Score: 0.095 (lowest).
Diversity Score: 0.701 (highest).
Novelty Score: 0.347.

google/pegasus-xsum:
ROUGE Scores: Lower than most other models.
BLEU Score: 0.080 (lowest).
Diversity Score: 0.230 (lowest).
Novelty Score: 0.424.

google/pegasus-cnn_dailymail:
ROUGE Scores: Strongest overall performance.
BLEU Score: 0.271 (highest).
Diversity Score: 0.596 (second highest).
Novelty Score: 0.439 (highest).

allenai/led-large-16384:
ROUGE Scores: Very strong performance across the board.
BLEU Score: 0.355 (second highest).
Diversity Score: 0.495 (third highest).
Novelty Score: 0.418.

allenai/led-large-16384-arxiv:
ROUGE Scores: Good, but not the best.
BLEU Score: 0.201 (third highest).
Diversity Score: 0.286 (fourth highest).
Novelty Score: 0.424 (second highest).

Summary:
Best ROUGE Scores: allenai/led-large-16384
Best BLEU Score: google/pegasus-cnn_dailymail
Best Diversity: facebook/bart-large-cnn
Best Novelty: google/pegasus-cnn_dailymail
Overall Recommendation:
If ROUGE scores are the most important to you, allenai/led-large-16384 is the best choice. It performs well across all ROUGE metrics. However, if BLEU score and novelty are more critical, google/pegasus-cnn_dailymail would be preferred. It has the highest BLEU and novelty scores.

For a well-rounded performance across different metrics, google/pegasus-cnn_dailymail seems to be the best overall performer based on the evaluation metrics you provided.

# for splitting summary into section and then combing them this method
# but after seing the actual outputs we find that the BART cnn large is best and gives
#correct contextual summary.

In [ ]:
# with simple summarizing of the text

In [5]:
from transformers import BartTokenizer, BartForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration, LEDTokenizer, LEDForConditionalGeneration
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from collections import Counter
import re

# Define models and tokenizers
models = {
    'facebook/bart-large-cnn': (BartTokenizer, BartForConditionalGeneration),
    'google/pegasus-cnn_dailymail': (PegasusTokenizer, PegasusForConditionalGeneration),
    'allenai/led-large-16384': (LEDTokenizer, LEDForConditionalGeneration),
    'allenai/led-large-16384-arxiv':(LEDTokenizer, LEDForConditionalGeneration)
}

def get_model_and_tokenizer(model_name):
    tokenizer_class, model_class = models[model_name]
    tokenizer = tokenizer_class.from_pretrained(model_name)
    model = model_class.from_pretrained(model_name)
    return tokenizer, model

def summarize_text(model_name, text, max_length=1000, min_length=500):
    tokenizer, model = get_model_and_tokenizer(model_name)
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=4)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def calculate_rouge_scores(reference, summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, summary)
    return scores

def calculate_bleu_score(reference, summary):
    reference_tokens = reference.split()
    summary_tokens = summary.split()
    bleu_score = sentence_bleu([reference_tokens], summary_tokens)
    return bleu_score

def calculate_diversity(summary):
    words = summary.split()
    word_counts = Counter(words)
    return len(word_counts) / len(words) if words else 0

def calculate_novelty(summary, reference):
    summary_words = set(summary.split())
    reference_words = set(reference.split())
    novel_words = summary_words - reference_words
    return len(novel_words) / len(summary_words) if summary_words else 0

# Example text and reference
text = """Hello all, my name is Krushnayak and welcome to my YouTube channel. So guys before the start of every new year, I usually plan that what all skill sets I really need to add in my bucket list so that I'll be able to teach you all. I will be also able to show you that how specifically it is used in industries. In 2023, you know, I had targeted various MLops platform and if you probably see my videos in my YouTube channel, I've covered a lot of end to end projects. I have covered a lot many MLops platform tools, you know, and I've shown each and everything, which was really beneficial for many people out there who really want to make successful career transition into the data science industry. And MLops, I think in 2024 also it will play a very important role since right now LLM applications are basically coming up, you know, generative AI is there. So many tools in generative AI have actually come frameworks, libraries and many more things. So MLops will still be there in 2024. But my main aim in 2023 was to understand all about different MLops platform and try to create videos, try to upgrade myself with respect to all those skill sets and provide those videos to you. This was very beneficial. Many people were able to make successful career transition. Now in 2024, I'm also going to focus. See in 2023 also from past three to four months, I'm really much focused in generative AI solving amazing use cases, trying to see multiple frameworks, trying to see multiple LLM models, how you can fine tune it, how you can probably use it in business use cases. And based on that, I'm also creating projects. But majorly in 2024, right, the weightage between MLops and generative AI, I will try to put my most of the weightage in understanding different, different frameworks, cloud platforms, techniques in business use cases in the field of generative AI. So in this video, like if you are also interested and obviously you can see the growth in generative area, how it is happening every other day, some new things are actually coming, right? Let it be models, let it be LLM models, the recent launch of Google Gemini. I know that Gemini did not provide you the right kind of demo video. But other than that, you have Mistral 7B, you have OpenAI, GPD for turbo, right? All these models are specifically coming. Now people are using this in creating amazing AI applications. And you can see there are a lot of startups from image to text, text to image. There are so many different things. I was just seeing one AI platform today, you know, it is basically called as Pickalabs. You just give a prompt and it will try to create an amazing video for you, right? And just imagine you don't even require a video editing any person over there, right? And this is super easy, right? When you probably see this kind of application. So 2024, I'm going to completely focus most of my, if I say 100% from 100%, I'm going to focus 60 to 70% in generative AI and 30% with respect to MLOps platform. And don't worry, every videos that I probably learn will be coming in the form of YouTube channel. But in this video, if you are also really interested to understand about generative AI, I've created an amazing roadmap to learn generative AI in 2024. And this will be specifically for two kinds of roles, which I'm actually going to discuss. And this roadmap, trust me, it has some prerequisites. I'm also going to provide you some videos over here, because if you just cover those prerequisites, that will be more than sufficient to start with generative AI. Yeah. And this is for all those people who are really interested to work in the data field, right? Data analytics field. If you're a developer, if you probably want to work into the generative AI, I think you don't even require this prerequisite. And I'll also call out that specific information over here in my GitHub. Once I probably show you the specific roadmap. So let me go ahead and let me share my screen. So this is the roadmap to learn generative AI. So here you can probably see everything is there. I've provided videos link, I have everything, the prerequisites that is actually required. Everything is mentioned over here. And please make sure that you fork this repository, keep a star on it, because 2024, I am going to really create a lot many things over here. It'll be quite amazing, right? Many, many things will be coming with respect to projects, with respect to frameworks, with respect to what are things I'm going to specifically cover in Lanch in everything, right? So let's start and let's understand this particular roadmap. Before you go ahead, please make sure that you hit like for this particular video will target at 2000 likes at least, you know, because this will be super beneficial because here you're going to get all the videos, all the materials, everything in front of you with respect to the prerequisites. Now I will talk about two different profiles over here. One profile is that if you want to get into the data analytics industry, and let's say you want to probably start if you're starting from scratch, okay, starting from scratch, or from past two years, you're also already in the data analytics field, you're learning about data science, what kind of upgradation you can basically do. The second type of people will be core developers, right? So in my company also, we have used generative AI, we are creating support systems, we are creating assignment, Q&A systems, many more things are there. Right now those developers are specifically creating and they don't have need to have all the knowledge with respect to some kind of prerequisites as such directly they can start with generative AI. So I will be talking about both of them step by step will try to understand. So the roadmap to learn about generative AI in 2024 is that first of all, you start with one programming language. I will again prefer Python programming language because trust me any LLM models that is probably coming from hugging face to open AI to Mistral everywhere Python SDK is there, right? So Python SDK with the help of Python programming language, you will be able to access those API's, you'll be able to probably work on them, you will be able to probably implement any kind of applications. And not only that, you'll also be able to deploy because they are good cloud platforms that are specifically come with respect to that. Okay, so here, Python programming language, I have given this particular link, see, I've already created Python programming language in English in Hindi, we'll be able to probably find out all this particular videos. I've already shown you this particular videos, good videos, if you probably see there are millions and millions of views with respect to this all videos. Again at the end of the day, this will be super beneficial for everyone out there, right? And not only this Python, if you just want to also become a Python programmer, everything is covered from modular coding to inheritance to oops constant, everything is probably explained along with materials. So this is the first thing. Now here, in third and fourth, I have also added some frameworks playlist, right? Like what like Flask playlist, fast API tutorials and all, right? So these are frameworks, streamlet is one framework, different different frameworks you can specifically use, okay? It is up to you. See, I've created videos on streamlet also I've created videos on Gradio also, but I've just given two frameworks over here Flask and fast API to just give you an idea. If you search for streamlet in my YouTube channel, you'll be able to get those videos also. So Python is the first thing, okay? Usually if you are from another like other like core, if you are directly a developer, you have working in applications, you're creating like full stack web developer, you're having those kind of position. And let's say someone comes and tells you, okay, go ahead and apply this generative way. I think in our project, try to create a chat bot, try to create something. At that time, you don't require Python, whatever will be a core language like JavaScript or you can go ahead with that because that kind of SDK is also provided. But this is specifically with respect to the data field. Now coming to the next thing, see, over here, the roadmap that I'm talking about generative AI, this is with respect to LLM models, I'm not talking about large image models, okay? What I feel is that computer vision, large image models is also good. But the kind of kind of update, upgradation that are specifically happening is with respect to LLM models. This is the one thing that I'm talking about this roadmap, I really want to make it for understanding like how you can become better in generative AI in the field of LLM applications, okay, large language model. Whenever I say large language model, I'm basically talking about NLP. Okay? I am not exploring much in large image models because computer vision, I too don't have that much interest. So in my YouTube channel, so you'll be finding videos that I have not uploaded much with respect to computer vision, okay? But I'm really interested in NLP. So after completing Python, you will basically be having the basic machine learning natural language processing. Again, this is basically the prerequisites, okay? And here I've given, I've taken already natural language processing live session where I've covered in five days all these topics, YNLP, one-hot encoding, bag of words, TF-IDF, word to beck, average word to beck, and there are a lot many topics that I've actually covered, which you will be able to find over here from day one to day five. So if I probably go ahead and open this particular video here from day one to day five, you'll be getting everything like word embedding, C bow, what is skip gram, word to beck. This five video will be able to help you understand because see, this is a very good roadmap, a prerequisite, a short roadmap. I'm not going to make a very big roadmap because this concept is basically used so that you understand them. What is vectors? You know, how we convert text into a numerical variable. All these things will be important for your interview because in the interview, they'll not directly are generated way, I think. They'll first of all see how good your basics is, right? So considering this, here I could add more topics over here based on this day one to day five, but your task is as a prerequisite is to basically see this day one to day five. And this also includes practical implementation. Now the third day, third part is that you need to cover basic deep learning concepts. Now when I say basic deep learning concepts, this is something related to ANN, like how does a multilayer neural network work? What is perceptron? What is forward propagation, backward propagation, activation function, loss function optimizers? What is weight initialization techniques? What is vanishing gradient problem? Everything right over here. Again I have not written much topic so that you don't look, it does not look very big for you and you don't get demotivated. But all these concepts, again I have made a live playlist in my community series that is regarding day one to day five. So if you probably open this link, okay. So here you'll be able to see day one to day five. After day five, if I see this deep learning concept. So what I will do, I will open my YouTube channel and I'll write Krishnayak live deep learning, okay. So I will update those link over there. And here you have, right, so this is basically your live deep learning sessions, okay. So here you can probably see this day one to day five. This is basically the prerequisite, right. All the important things, how does forward propagation work? What is chain rule of derivatives? What exactly is optimizers? What exactly is loss function? What is forward propagation, backward propagation? The CNN I have actually implemented and shown in the practical way. So this link I will try to update it over here, right. So in basic deep learning concept, this is must. Because unless you don't understand that, the further topics will not be able to understand. This is the smartest way that I've actually created a roadmap where unnecessary jargons need not be added, okay. Then after you complete this, then we go to the advanced NLP concept. Now advanced NLP concept is nothing but day six to last video. Which one? If you probably click this link, here you'll be able to see six to entire tilt transformer. So here you can see day six, day seven. Here I've discussed about RNN. Here I've discussed about back propagation in RNN, LHTMRNN, word embedding, LHTM practical implementation, advanced LNM series, bidirectional LHTM, transformers, encoder, decoder. Everything is basically covered in all the specific videos. So if you are able to cover this from day six to probably 13th or 14th video, all these topics will be easily covered. If you are not able to find some topic, go and search for that topic with my name over there. I have a lot of videos. I have 1800 plus videos in my YouTube channel. When I've covered everything and this is what I've done from past three years, all the basics concepts have made strong so that you can clear the interview. There is no query that you cannot clear the interview guys. The kind of content that I've put in my YouTube channel is completely from free scratch, everything from basics so that you learn any advanced thing. It will be very much easy for you to crack the interview. So that is what my advanced NLP concept says where my main aim was to cover the transformer because after that, whenever you start your generative AI, that basically means most of your models are in the form of transformers or BERT, right? So here you have GPT-4, Mistral 7B. So here is where you probably start your journey. This is what is the prerequisite till here, right? And that is the reason I've created this prerequisite in this specific way. And it is simple. It's more about learn to the point, right? Don't waste your time much. Learn to the point, see some practical implementation because I don't think so this kind of practical implementation you will be doing in the industry also. But yes, in the interview, they may ask you now for those people who are directly developer, they can also jump in this particular topic, right? They can directly start from here because at the end of the day, they are very good at development, you know, who are full stack web developer who are working in the software engineering field, they can directly use these APIs and they can implement it. Now if I talk about some important things over here, right? Starting the journey towards generative AI. You need to really be very much open minded in probably doing a research on all these models, right? So Mistral 7b, Lama, Lama index, hugging face, open source libraries, Google Palm model and all, right? I've written over here because there are a lot of updates that are coming with respect to the specific models. And I'll tell you GPT-4 is must, hugging face is must, Lama, Mistral 7b, Mistral 7b has now recently come up with one amazing model. Again, we say it as a 87 cross B, right? Something like that. I'll be talking more about those models as we go ahead, right? And also try to see open source LNM models because with all this help of these models, you know, you can develop any business use cases that are specifically related to NLP, right? NLP, natural language processing, any use cases, chatbot to text summarization to documentation, anything, quiz, anything. So that is specifically required for the companies, you will be able to do this, right? And that is the reason why many companies are specifically using it. Large image models also you can use because all this large image models they'll be providing your diffusion models, they'll be providing your Dali, right? And based on that, you can do all further on top of it. But really, I'm focusing more on LLMs right now. So the first thing is OpenAI. It has amazing documentation link. Have already created videos, good playlist of videos over here, you can definitely look it out. And this documentation you really need to be good at. And the best thing about this documentation in OpenAI, it provides a good amazing things, right? OpenAI, start with this. Videos are given, you can definitely refer it. This year, I'm going to target LangChain like anything. LangChain, already if you go and see my YouTube playlist, they are on 10 to 12 videos. But LangChain is one amazing library that acts as a wrapper on top of OpenAI. It can also use hugging face. It has so many different functionalities to create all the LLM models, not only this. So LangChain also provides deployment techniques like LangServ, chain rest as rest APIs. This is nothing but it is entirely called as LangSmith. So you can probably see this is your LangChain application. It may be in Python, it may be in JavaScript. On top of that, you will be creating some template, you will be doing the deployment in the form of APIs by using this LangSmith. So going forward, I really want to probably see this entire tool and deployment techniques also and it is very much it is recently been announced, right? Why I like LangChain is that because it has so many different functionalities over here, right? Different functionalities from chatbots to prompts to modules to probably if you go and see right, what is LangServ will be releasing a hosted version of LangServ from one click deployment. Just imagine just one click deployment. All these things we are going to explore like anything as we go ahead, right? In LLMs, what exactly is unsync API, dissolve, prompting, most of the videos that I have already created, but every day I see something is getting added in this documentation, right? Over here only, if you probably see with respect to retrieval, right? In document loader, like which all documents will be able to load like CSV, file directory, HTML, JSON, markdown, PDF, document transformer, how do you split the documents and all, everything. But most of this everything will be combined in the form of a project in the end to end project where I do the deployment in an amazing way, right? All those things will basically be covered and here is a detailed video. After I get a good expertise on LangChain, I will also be starting with Chainlit. So Chainlit is another one documentation which looks very good over here, right? Good you will be able to do everything with the help of LangChain, Lama index, here also it is supported. You will also be able to do the deployment, right? Everything is there. You can do it in AWS, Azure, Google route, Replate, render, fly.io, hugging face spaces, okay? So once you probably cover this, right now in these two years you can probably target OpenAI and LangChain and Chainlit, okay? That is what I am actually going to do and based on this I will try to create a lot of videos, okay? LangChain, again you can interact with hugging face models, Lama, Mistral 7b, anything as such, whatever you want, okay? The next topic that you really need to focus is about vector databases and vector stores. Now vector database will play a very important role and again this will again be a part of LangChain itself because in LangChain it provides you functionalities. Some of the vector databases that I have already explored is like ChromaDB, FIAS. It is FIAS vector database is nothing but which makes use of Facebook AI similarity search library, right? So this is coming from Facebook. LandDB vector database based on the Lange data format. So there is a format which is called as Lange data set and again there you will be able to apply similarity search. Cassandra DB for storing vectors, Cassandra DB is also amazing, MongoDB is also amazing, right? At the end of the day, if you have any text you really want to convert that into vectors and since it is for the performance sake you really need to store it in some kind of vector database so that you can retrieve it and enjoy it. Then finally after doing this you have to probably do the deployment of LLM projects and this is what I am going to target. Within one month you will find all these videos in my YouTube channel, right? In AWS, Azure, LangSmith, LangSpa, HuggingFace, wherever you want to do the deployment. And finally guys if you really want to go ahead and check like how does, what is GenitiveAI, whether GenitiveAI is there for nano. I have given the course link over here, okay? This is a free community course where everybody can probably apply for it. You will find all the videos, materials and all and it is live series. Right now we are in day 5, day 6. So please go ahead and check it out, it is completely for free. See whether GenitiveAI is for you or not. And I hope if you are already in the data field I feel you really need to go towards GenitiveAI, right? At the end of the day. Because for the interview sake, yes, basic questions will be asked. But when you implement things in the production level, this all will be very much handy. And this is what is my main aim in 2024. I hope you like this particular video. Yes, please go ahead. Right now this is in private, I will make it public. So let me go ahead and make it public. So I will change it to public so that you can also access it. Make this repository public. And now one final thing is that I will also update this specific link of deep learning. So I hope you like this particular video. This was it from my side. I will see you all in the next video. Have a great day. Thank you and all. Take care. Bye bye."""

# Evaluate each model
results = {}

for model_name in models.keys():
    print(f"Evaluating model: {model_name}")
    # Generate summary
    formatted_summary = summarize_text(model_name, text)
    
    # Evaluate summary
    rouge_scores = calculate_rouge_scores(text, formatted_summary)
    bleu_score = calculate_bleu_score(text, formatted_summary)
    diversity_score = calculate_diversity(formatted_summary)
    novelty_score = calculate_novelty(formatted_summary, text)
    
    # Store results
    results[model_name] = {
        'ROUGE Scores': rouge_scores,
        'BLEU Score': bleu_score,
        'Diversity Score': diversity_score,
        'Novelty Score': novelty_score
    }

# Print comparative results
for model_name, metrics in results.items():
    print(f"\nResults for {model_name}:")
    print("ROUGE Scores:", metrics['ROUGE Scores'])
    print("BLEU Score:", metrics['BLEU Score'])
    print("Diversity Score:", metrics['Diversity Score'])
    print("Novelty Score:", metrics['Novelty Score'])


Evaluating model: facebook/bart-large-cnn
Evaluating model: google/pegasus-cnn_dailymail


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluating model: allenai/led-large-16384
Evaluating model: allenai/led-large-16384-arxiv

Results for facebook/bart-large-cnn:
ROUGE Scores: {'rouge1': Score(precision=0.8385542168674699, recall=0.0878566018682151, fmeasure=0.1590493601462523), 'rouge2': Score(precision=0.4855072463768116, recall=0.05075757575757576, fmeasure=0.09190672153635118), 'rougeL': Score(precision=0.5927710843373494, recall=0.062105528906841706, fmeasure=0.11243144424131626)}
BLEU Score: 6.479680688989115e-05
Diversity Score: 0.5211970074812967
Novelty Score: 0.27751196172248804

Results for google/pegasus-cnn_dailymail:
ROUGE Scores: {'rouge1': Score(precision=0.6336336336336337, recall=0.05326937642009594, fmeasure=0.09827666511411272), 'rouge2': Score(precision=0.20180722891566266, recall=0.01691919191919192, fmeasure=0.03122087604846226), 'rougeL': Score(precision=0.37537537537537535, recall=0.03155768745266347, fmeasure=0.058220773171867725)}
BLEU Score: 3.0878233154505345e-07
Diversity Score: 0.12925170

In [ ]:
Summary of Metrics
Facebook BART-Large-CNN

ROUGE Scores:
ROUGE-1: Precision: 0.839, Recall: 0.088, F-measure: 0.159
ROUGE-2: Precision: 0.486, Recall: 0.051, F-measure: 0.092
ROUGE-L: Precision: 0.593, Recall: 0.062, F-measure: 0.112
BLEU Score: 6.48e-05
Diversity Score: 0.521
Novelty Score: 0.278
Google Pegasus-CNN/DailyMail

ROUGE Scores:
ROUGE-1: Precision: 0.634, Recall: 0.053, F-measure: 0.098
ROUGE-2: Precision: 0.202, Recall: 0.017, F-measure: 0.031
ROUGE-L: Precision: 0.375, Recall: 0.032, F-measure: 0.058
BLEU Score: 3.09e-07
Diversity Score: 0.129
Novelty Score: 0.289
AllenAI LED-Large-16384

ROUGE Scores:
ROUGE-1: Precision: 0.060, Recall: 0.015, F-measure: 0.024
ROUGE-2: Precision: 0.006, Recall: 0.002, F-measure: 0.002
ROUGE-L: Precision: 0.059, Recall: 0.015, F-measure: 0.023
BLEU Score: 0.000443
Diversity Score: 0.019
Novelty Score: 0.421
AllenAI LED-Large-16384-Arxiv

ROUGE Scores:
ROUGE-1: Precision: 0.787, Recall: 0.071, F-measure: 0.130
ROUGE-2: Precision: 0.363, Recall: 0.033, F-measure: 0.060
ROUGE-L: Precision: 0.579, Recall: 0.052, F-measure: 0.095
BLEU Score: 1.71e-05
Diversity Score: 0.199
Novelty Score: 0.056
Analysis
ROUGE Scores:

Best ROUGE-1: Facebook BART-Large-CNN (0.839)
Best ROUGE-2: Facebook BART-Large-CNN (0.486)
Best ROUGE-L: Facebook BART-Large-CNN (0.593)
BLEU Score:

Best BLEU Score: Facebook BART-Large-CNN (6.48e-05)
Diversity Score:

Best Diversity: Facebook BART-Large-CNN (0.521)
Novelty Score:

Best Novelty: AllenAI LED-Large-16384 (0.421)
Conclusion
Facebook BART-Large-CNN appears to be the best overall model based on ROUGE scores, BLEU score, and diversity. It has the highest ROUGE scores across all metrics, which indicates it performs well in terms of content overlap and similarity to reference summaries. Although its novelty score is lower compared to AllenAI LED-Large-16384, it provides a more comprehensive summary in terms of ROUGE and BLEU evaluations.

AllenAI LED-Large-16384 has the highest novelty score but has significantly lower performance in ROUGE and BLEU metrics, which may suggest it generates more unique content but with less overall alignment to the reference.

In summary, Facebook BART-Large-CNN is recommended if the focus is on overall summary quality and alignment with the reference text.